In [ ]:
# Setup and Imports

# %reload_ext : Reloads an IPython extension by its module name.
%reload_ext autoreload
# %autoreload 2 : Reloads all modules (except those excluded by %aimport)  
#  every time before executing the Python code typed.
%autoreload 2

# %TEXT : code in this format is called a "magic function" 
#  https://stackoverflow.com/questions/43027980/purpose-of-matplotlib-inline/43028034

import os
import sys
import time

import matplotlib.pyplot as plt
# Sets backend of matplotlib to the 'inline' backend
%matplotlib inline
import numpy as np

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *
from learning_utilities import *

# Miscillaneous Code
#### Function to detect where you are at running the code

In [ ]:
# BAD FILE LOCATOR
# Automatically tells where in the code you are

# 1) raw -> raw_Ntb: extract_channel
# 2) raw_Ntb -> thumbnail_Ntb: rescale
# 3) thumbnail_Ntb -> thumbnail_NtbNormalized: normalize_intensity
# 4) Compute transforms using thumbnail_NtbNormalized: align + compose
# 5) Supply prep1_thumbnail_mask
# 6) prep1_thumbnail_mask -> thumbnail_mask: warp
# 7) raw_Ntb -> raw_NtbNormalizedAdaptiveInvertedGamma: brightness_correction
# 8) Compute prep5 (alignedWithMargin) cropping box based on prep1_thumbnail_mask
# 9) raw_NtbNormalizedAdaptiveInvertedGamma -> prep5_raw_NtbNormalizedAdaptiveInvertedGamma: align + crop
# 10) thumbnail_NtbNormalized -> prep5_thumbnail_NtbNormalized: align + crop
# 11) prep5_raw_NtbNormalizedAdaptiveInvertedGamma -> prep5_thumbnail_NtbNormalizedAdaptiveInvertedGamma: rescale
# 12) Specify prep2 (alignedBrainstemCrop) cropping box
# 13) prep5_raw_NtbNormalizedAdaptiveInvertedGamma -> prep2_raw_NtbNormalizedAdaptiveInvertedGamma: crop
# 14) prep2_raw_NtbNormalizedAdaptiveInvertedGamma -> prep2_raw_NtbNormalizedAdaptiveInvertedGammaJpeg: compress_jpeg

# np.setdiff1d(list1,list2) # Get difference between two lists
# metadata_cache['valid_filenames_all'][stack]  # To get all valid filenames

def where_am_i():
    '''
    where_am_i will iterate through each of the 14 steps of preprocessing, checking that ALL output files 
    have been generated sucessfully. For any stage where files are missing or corrupted, this function will 
    return a list of the filepaths & filenames of the images that are not in the output.
    where_am_i() returns [bad_fp_list, bad_file_list].
    '''
    
    # Throws an error if you don't set Max Pixels to around 2 Billion (yikes)
    Image.MAX_IMAGE_PIXELS = 2000000000
    # Iterates through every possible filename as defined in metadata_cache
    #                                           stack must be defined
    bad_fp_list = [] # Full filePaths
    bad_file_list = [] # FileNames
    
    #   Dic = {'step#': [prep_id, resol, version]}
    stepDic = {'1':[None,'raw','Ntb'], '2':[None,'thumbnail','Ntb'], '3':[None,'thumbnail','NtbNormalized'],\
               '4':[None,'???','???'], '5':[1,'thumbnail','mask'], '6':[None,'???','???'], \
               '7':[None,'raw','NtbNormalizedAdaptiveInvertedGamma'], '8':[5,'thumbnail','mask'], \
               '9':[5,'raw','NtbNormalizedAdaptiveInvertedGamma'], '10':[5,'thumbnail','NtbNormalized'], \
               '11':[5,'thumbnail','NtbNormalizedAdaptiveInvertedGamma']}

    
    # For checking each individual step of the pipeline as outlined above
    for step in range (2,12):
        stepStr = str(step)
        stepCompleted = True
        
        print 'Checking for completion of step '+stepStr
        t = time.time()
        
        if stepDic[stepStr][1]=='???':
                stepCompleted = False
                print 'Output files unknown. No idea.'
                print ''
                continue
        
        for fn in metadata_cache['valid_filenames_all'][stack]:
            
            # Record the filepath from every filename + info from the "step dictionary"
            fp = DataManager.get_image_filepath_v2(stack=stack, prep_id=stepDic[stepStr][0], \
                                     resol=stepDic[stepStr][1], version=stepDic[stepStr][2], fn=fn)
            
            fp_to_test = fp
            try:
                img = Image.open(fp_to_test) # open the image file
                img.verify() # verify that it is, in fact an uncorruted image
            except (IOError, SyntaxError) as e:
               # print('Bad file:', fp_to_test) # Print out the names of corrupt files
                bad_fp_list.append(fp_to_test) # Add bad filepaths to bad_filepath_list
                bad_file_list.append(fn) # Add bad files to bad_file_list
                stepCompleted = False
        #metadata_cache['valid_filenames_all']['MD662']
        
        if stepCompleted:
            print 'Step '+stepStr+' completed!'
            print 'done in', time.time() - t, 'seconds' 
            print ''
        else:
            print '*****     Bad files found!     *****\n'
            print 'Step '+stepStr+' NOT complete. Rerun necessary parts of the code'
            print '`bad_fp_list` and `bad_file_list` created, each contain info on each incomplete files.'
            print 'done in', time.time() - t, 'seconds' 
            print ''
            if stepDic[stepStr][0] is None:
                print '`'+stack+'_'+stepDic[stepStr][1]+'_'+\
                        stepDic[stepStr][2]+'` was not generated properly'
            else:
                print '`'+stack+'_prep'+str(stepDic[stepStr][0])+'_'+stepDic[stepStr][1]+'_'+\
                        stepDic[stepStr][2]+'` was not generated properly'
            break
            #continue
    return bad_fp_list, bad_file_list

#print where_am_i.__doc__
[bad_fp_list, bad_file_list] = where_am_i()

print len(bad_file_list),' bad files'

# STEP 1
### raw (.jp2) -> raw_Ntb (.tif)

# STEP 2
### raw_Ntb -> thumbnail_Ntb
rescale

# STEP 3
### thumbnail_Ntb -> thumbnail_NtbNormalized
normalize_intensity

# STEP 4
### Compute transforms using thumbnail_NtbNormalized
align + compose

# STEP 5
### Supply prep1_thumbnail_mask

# STEP 6
### prep1_thumbnail_mask -> thumbnail_mask
warp

# STEP 7
### raw_Ntb -> raw_NtbNormalizedAdaptiveInvertedGamma
brightness_correction

# STEP 8
### prep1 -> prep5
Compute prep5 (alignedWithMargin) cropping box based on prep1_thumbnail_mask

# STEP 9
### raw_NtbNormalizedAdaptiveInvertedGamma -> prep5_raw_NtbNormalizedAdaptiveInvertedGamma
align + crop

# STEP 10
### thumbnail_NtbNormalized -> prep5_thumbnail_NtbNormalized
align + crop

# STEP 11
### prep5_raw_NtbNormalizedAdaptiveInvertedGamma -> prep5_thumbnail_NtbNormalizedAdaptiveInvertedGamma
rescale

# STEP 12
### Specify prep2 (alignedBrainstemCrop) cropping box


# STEP 13
### prep5_raw_NtbNormalizedAdaptiveInvertedGamma -> prep2_raw_NtbNormalizedAdaptiveInvertedGamma
crop

# STEP 14
### prep2_raw_NtbNormalizedAdaptiveInvertedGamma -> prep2_raw_NtbNormalizedAdaptiveInvertedGammaJpeg
compress_jpeg